In [12]:
import torch
from torchvision import transforms
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from torch.utils.data import Dataset, DataLoader, random_split
from torch import optim
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
dataframe = pd.read_csv("dataset/fashion-mnist_train.csv", delimiter=',')
dataframe = dataframe[dataframe['label'].isin([0, 2, 3])]

In [3]:
dataframe.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,1,0,0,0,0,...,164,177,163,0,0,1,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
class MnistDataset(Dataset):
    def __init__(self, pd_table: pd.DataFrame, normalize: bool = True, encode_y = True):
        self.data = pd_table
        self.X = self.data.drop(columns='label')
        self.Y = self.data[['label']]
        if normalize:
            self.X = pd.DataFrame(StandardScaler().fit_transform(self.X), columns=self.X.columns)
        if encode_y:
            encoder = OneHotEncoder()
            self.Y = torch.Tensor(encoder.fit_transform(self.Y).toarray())

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index: int):
        image = torch.Tensor(self.X.iloc[index])
        return image, self.Y[index]

In [5]:
dataset = MnistDataset(dataframe)

In [6]:
n_input = 784
linear_model1 = nn.Sequential(nn.Linear(n_input, 256),
                              nn.ReLU(),
                              nn.Linear(256, 3),
                              nn.Softmax()                    
)

linear_dropout_model1 = nn.Sequential(nn.Dropout(0.1),
                      nn.Linear(n_input, 256),
                      nn.ReLU(),
                      nn.Linear(256, 3),
                      nn.Softmax())

linear_model2 = nn.Sequential(nn.Linear(n_input, 512),
                              nn.ReLU(),
                              nn.Linear(512, 256),
                              nn.ReLU(),
                              nn.Linear(256, 3),
                              nn.Softmax()                    
)

linear_dropout_model2 = nn.Sequential(
                            nn.Dropout(0.1),
                            nn.Linear(n_input, 512),
                            nn.ReLU(),
                            nn.Linear(512, 256),
                            nn.ReLU(),
                            nn.Linear(256, 3),
                            nn.Softmax()                    
)



names = ["Модель 1 без дропаута", "Модель 1 с дропаутом", "Модель 2 без дропаута", "Модель 2 с дропаутом"]
models = [linear_model1, linear_dropout_model1, linear_model2, linear_dropout_model2]

In [7]:
train, test, validation = random_split(dataset, [0.6, 0.2, 0.2])
batch_size = 8
train, validation = DataLoader(train, batch_size=batch_size), DataLoader(validation, batch_size=batch_size)


In [8]:
def train_model(model, train_loader, val_loader, learning_rate=0.001, n_epoch=10, loss_function=nn.CrossEntropyLoss(),
                print_epoch=True):
    start_time = time.time()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    for epoch in range(n_epoch):
        for img, label in train_loader:
            out = model(img)
            train_loss = loss_function(out, label)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

        summary_loss = 0
        for img, label in val_loader:
            out = model(img)
            summary_loss += loss_function(out, label)

        validation_loss = summary_loss/len(validation)
        
        #Ранняя остановка
        if epoch != 0 and prev_validation_loss <= validation_loss:
            break

        
        prev_validation_loss = validation_loss
        if print_epoch:
            print("Epoch: %d, Training loss: %f, Validation loss: %f" % (epoch, float(train_loss), float(validation_loss)))
    return validation_loss, time.time() - start_time

In [9]:
losses = []
times = []
for name, model in zip(names, models):
    print(name)
    loss, t= train_model(model, train, test, print_epoch=False)
    losses.append(loss)
    times.append(t)
    print(loss, t)
best_model_index = losses.index(max(losses))

Модель 1 без дропаута


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(5.0043, grad_fn=<DivBackward0>) 107.68285036087036
Модель 1 с дропаутом
tensor(5.0173, grad_fn=<DivBackward0>) 107.55097961425781
Модель 2 без дропаута
tensor(5.0153, grad_fn=<DivBackward0>) 133.0048999786377
Модель 2 с дропаутом
tensor(5.0225, grad_fn=<DivBackward0>) 148.6459186077118


In [10]:
best_model_index = losses.index(max(losses))

In [11]:
best_model = models[best_model_index]
true_values = 0
for image, label in test:
    predicted = best_model(image)
    if torch.argmax(predicted) == torch.argmax(label):
        true_values += 1
print(f"_______\n{true_values/len(test)}")

_______
0.9325


AttributeError: 'Subset' object has no attribute 'shape'